Homework 5

Copy this notebook. Rename it as: YOURNAME-HW5-streams 

with your name replacing YOURNAME.

Upload your completed jupyter notebook to elearning site as your homework submission. You can put this notebook on your github.

5.1  Register for a stream of Twitter data

5.2  Create a bloom filter classifying two days worth of twitters  ( after removing stop words and urls )

5.3  For another days worth of twitter data find the previous twitters that match in the bloom filter
(This means get two days of data in one file or directory , use that data for training the bloom filter, capture a different days data in a different file ( or do it in real time)and capture the match output then running the new twitter data through the filter.

5.4 Plot a historgram of matches for each twitter in 5.3

For the 4-5 grade.- Submit in a separate notebook - YourNAME-Homework5-Supplement

1. Use a different machine learning training algorithm
2. Make a continous feed where you take two days of data and match the incoming stream ( do this for 5 days windowing the filter data)
3. Find new trends in the twitter feed (daily or hourly)
4. Or some other streaming exploration of your choosing

In [24]:
%%writefile twitter_streamingSB.py
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

#Variables that contains the user credentials to access Twitter API 
access_token = "925146948012822528-BixZTf8fAkM4s5j89b7wT8rDm1FVALs"
access_token_secret = "EjsuYJuhHQK0WnIkxnIrl8qQXCZx6cgHd1yygsejn7njU"
consumer_key = "1PUalMqq9WpgZ5o6bJUtPKJNJ"
consumer_secret = "U1N73shHsc27QFLNRCi0u9KjayMlupMSQc8nCA1ZZEAV5UMqQm"

class StdOutListener(StreamListener):

    def on_data(self, data):
        print data
        return True

    def on_error(self, status):
        print status

#This is a basic listener that just prints received tweets to stdout.
if __name__ == '__main__':

    #This handles Twitter authetification and the connection to Twitter Streaming API
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    stream = Stream(auth, l)

    #This line filter Twitter Streams to capture data by the keywords: 'apple', 'google', 'microsoft'
    stream.filter(track=['apple', 'google', 'microsoft'])
    

Overwriting twitter_streamingSB.py


In [25]:
%%cmd
python twitter_streamingSB.py > twitter_data_pat2.txt

Microsoft Windows [Version 10.0.15063]
(c) 2017 Microsoft Corporation. All rights reserved.

(C:\Users\AsusGo\Anaconda2) C:\Users\AsusGo\Downloads\big-data-python-class-master\Homeworks\Homework6>python twitter_streamingSB.py > twitter_data_pat3.txt

(C:\Users\AsusGo\Anaconda2) C:\Users\AsusGo\Downloads\big-data-python-class-master\Homeworks\Homework6>

Traceback (most recent call last):
  File "twitter_streamingSB.py", line 31, in <module>
    stream.filter(track=['apple', 'google', 'microsoft'])
  File "C:\Users\AsusGo\Anaconda2\lib\site-packages\tweepy\streaming.py", line 445, in filter
    self._start(async)
  File "C:\Users\AsusGo\Anaconda2\lib\site-packages\tweepy\streaming.py", line 361, in _start
    self._run()
  File "C:\Users\AsusGo\Anaconda2\lib\site-packages\tweepy\streaming.py", line 294, in _run
    raise exception
urllib3.exceptions.ReadTimeoutError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Read timed out.


In [1]:
import json
import pandas as pd

tweets_data_path = 'twitter_data_part2.txt'

tweets_data = []
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue
        
print len(tweets_data)

tweets = pd.DataFrame()
tweets['text'] = [tweet.get('text','') for tweet in tweets_data]
tweets['lang'] = [tweet.get('lang','') for tweet in tweets_data]
tweets_by_lang = tweets['lang'].value_counts()
tweets.head(10)

77983


,text,lang
0,Pier 32 Industry: Magikarp ♂ til 17:33:00(13m ...,et
1,https://t.co/SCk9CG19sy\n\n#DavyJones,und
2,RT @M_Madoka_com: 《Google+更新》\n11/12\n\n東京着きまし...,ja
3,Send money to his opponent @GDouglasJones - Tr...,en
4,when google celebrates his birthday!!\nশুভ জন্...,und
5,Riachuelo? Prefiro andrajo como indumentária.\...,pt
6,RT @CNET: Microsoft CEO tells iPad users to 'g...,en
7,&gt;&gt; https://t.co/LWgbwOPous,und
8,Dog Beach: Magikarp ♂ til 17:49:17(29m 49s). h...,da
9,حمّل الآن لعبة #كلمةـالسر #الجزءـ٢ #زيتونة أيف...,ar


In [77]:
#tweets.groupby(['lang']=='en')['text']
tweet_text=tweets.loc[tweets['lang'] == 'en', 'text']

print tweet_text.head(10)

3     Send money to his opponent @GDouglasJones - Tr...
6     RT @CNET: Microsoft CEO tells iPad users to 'g...
12    RT @KJKirby: The latest Microsoft Project Dail...
14    RT @vadpradub: #JudgeMoore accuser should be i...
15    Target: Kellogg’s Pop-Tarts Caramel Apple or P...
20    RT @realtycoinsinc: Notable and Influential An...
21    RT @seattlewebseo: Google Grants will help you...
22    RT @JoeFreedomLove: Here's how the House and S...
25    They called it "Dox?"\n\nThey. Called. It. Dox...
26    RT @JonathanYMusic: 🐉 SKYRIM THEME ~ METAL CO...
Name: text, dtype: object


Here I filtered the english tweets.

Elimination of stop words,special characters,urls and retweets.

In [142]:
tweet_text_without_urls = tweet_text.str.replace('http\S+|www.\S', ' ', case=False)
tweet_clean_data=tweet_text_without_urls.str.replace('@[A-Za-z0-9]|[^0-9A-Za-z \t]|RT+.*'+'[^A-Za-z0-9]+','', case=False)
#tweet_text_wo_urls_SpecChar=tweet_text_without_urls.str.replace('[^A-Za-z0-9]+','  ',case=False)
#tweet_text_wo_urls_SpecChar_RT=tweet_text_wo_urls_SpecChar.str.replace('RT+.*\:','', case=False)
#(@[A-Za-z0-9]+)+([^0-9A-Za-z \t])+(\w+:\/\/\S+)
print tweet_clean_data.head(5)

#tweet_text_wo_urls_stopwords_RT_SpecChar=tweet_text_wo_urls_stopwords_RT.str.replace('[^A-Za-z0-9]+','', case=False)
tweet_clean_data.str.split(' ', expand=True).stack().unique()
tweet_clean_data.str.split(' ', expand=True).stack().value_counts().head()

3     Send money to his opponent DouglasJones  Trump...
6                                                      
12                                               espc17
14                                                     
15                                Target Kelloggs PopTa
Name: text, dtype: object


          85552
the        5300
to         5192
Google     4655
a          4039
dtype: int64

Here,I removed the url links from all the rows by using replace()

I will elminate the stop words from the sub dataframe with the help of nltk package

In [143]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AsusGo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [146]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
tweet_clean_data_without_stopwords=tweet_clean_data.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#print tweet_clean_data_without_stopwords
#print tweet_text_without_urls_stopwords
#tweet_clean_data_without_stopwords.split(' ', expand=True).stack().unique()
#tweet_clean_data_without_stopwords.str.split(' ', expand=True).stack().value_counts().head()
new_tweets_clean_data=tweet_clean_data_without_stopwords.str.split(' ', expand=True).stack().unique()
tweet_clean_data_without_stopwords.str.split(' ', expand=True).stack().value_counts()
print new_tweets_clean_data
#result = ' '.join(tweet_text_without_urls_stopwords)
#print result.split()

[u'Send' u'money' u'opponent' ..., u'20132529m' u'19000' u'081416pm']


I elminated the stop words

In [159]:
import numpy as np
#new_tweets_clean_data.to_csv(r'c:\data\twitter_text_ouput1.txt', header=None, index=None, sep=' ', mode='a')
#np.savetxt('test.out', new_tweets_clean_data, delimiter=' ') 
np.savetxt(r'data/np.txt', new_tweets_clean_data, fmt='%s')

Storing the data of ndarray to a txt file using savetxt function

Applying Bloom Filter

In [165]:
from pybloom import BloomFilter
import os
import re

POST_DIR = 'data/'
# Read all my posts.
posts = {post_name: open(POST_DIR + post_name).read() for post_name in os.listdir(POST_DIR)}
# Create a dictionary of {"post name": "lowercase word set"}.
split_posts = {name: set(re.split("\W+", contents.lower())) for name, contents in posts.items()}

In [162]:
filters = {}
for name, words in split_posts.items():
    filters[name] = BloomFilter(capacity=len(words), error_rate=0.1)
    for word in words:
        filters[name].add(word)

References:

http://localhost:8890/notebooks/Downloads/big-data-python-class-master/Lectures/Lecture6-Streams/Streams%20Tutorial.ipynb

https://stackoverflow.com/questions/45395676/remove-a-url-row-by-row-from-a-large-set-of-text-in-python-panda-dataframe

https://stackoverflow.com/questions/29523254/python-remove-stop-words-from-pandas-dataframe

https://stackoverflow.com/questions/5843518/remove-all-special-characters-punctuation-and-spaces-from-string

https://stackoverflow.com/questions/32425334/splitting-a-string-in-a-python-dataframe

https://stackoverflow.com/questions/26594817/remove-and-rt-from-the-tweet

https://stackoverflow.com/questions/38557617/how-to-get-all-the-unique-words-in-the-data-frame

https://stackoverflow.com/questions/31247198/python-pandas-write-content-of-dataframe-into-text-file

